In this tutorial, we firstly recall the *port-Hamiltonian systems (PHS)* formalism in parallel with a description of the `pyphs.Core` object. Secondly, we build the `pyphs.Core` object associated with a two-ports serial RL circuit. Finally the Thiele-Small model of the loudspeaker is derived by connecting the RL circuit with a mass-spring-damper system. 
<!-- TEASER_END -->

The corresponding Python script `Core.py` can be found in the tutorials at `\pyphs\tutorials\`

# PHS structure of the Thiele-Small model

Steps are:
1. prepare the *governing equations* for the RL circuit,
2. instantiate a `pyphs.Core` *object*,
3. add the *components* (storage, dissipative and source),
4. define the *PHS structure* matrices (here the serial connection),
5. create the mass-spring-damper PHS as described in (1-4) and connect both `pyphs.Core` *objects*,
6. set the parameters values,
7. reduce the linear dissipative part,
8. simplify inversed symbols.

Bonus: Generate the associated LaTex document.

## 1. Governing equations
The two-ports RL circuit is described as follows:
* $x_L$ is the coil flux so that $v_L = \frac{\mathrm d x_L}{\mathrm d  t}$ is the coil voltage,  
* $w_R= i_R$ is the resistor current, 
* $y1=i_{\mathrm{1}}$ is the current at port #1 
* $u1=v_1$ is the voltage at port #1.
* $y2=i_{\mathrm{2}}$ is the current at port #2 
* $u2=v_2$ is the voltage at port #2.

For this tutorial, the constitutive laws are:
* the storage function (Hamiltonian) $$H(x_L)=\frac{x_L^2}{2L}$$ so that the (linear) coil current is $i_L=\frac{\partial H}{\partial x_L}=\frac{x_L}{L}$,
* the linear dissipation function $z_R(w_R)= R \,w_R = v_R$ with $v_R$ the resistor voltage. 

The Kirchhoff's laws for a serial connection read: 
* Kirchhoff's current law: $ v_L=-v_C-v_R-v_1-v2$,
* Kirchhoff's vcurrent law: $i_L = i_R = i_1 = i_2$.

This can be expressed in the *Port-Hamiltonian Systems* (PHS) formalism as:

\begin{equation}
\begin{pmatrix}
\frac{dx_L}{dt} \\
\hline
w_R \\
\hline
y_1 \\
y_2
\end{pmatrix}
= 
\left(
\begin{array}{c|c|cc|c}
0 &-1 &-1& -1 \\
\hline
1 &0 &0 &0 \\
\hline
1 &0 &0& 0 \\
1 &0& 0 &0
\end{array}
\right)
\begin{pmatrix}
\frac{dH}{dx_L} \\
\hline
z(w_R) \\
\hline
u_1 \\
u_2
\end{pmatrix}
\end{equation}

#### Physical parameters
* $L=500\times 10^{-3}$H, 
* $R = 10^2\Omega$.

## 2. Object instantiation

In this tutorial, we need only the core PHS structure implemented by the `pyphs.Core` object:

In [1]:
# import of the pyphs.Core class
from pyphs import Core

The instantiation takes an optional argument `label`:

In [2]:
# instantiate a pyphs.PHSCore object
RL= Core(label='my_core')

Now, `RL` is an instance of `pyphs.Core`:

In [3]:
isinstance(RL, Core)

True

In [4]:
RL.x

[]

In [5]:
RL.dims.w()

0

In [6]:
RL.M

Matrix(0, 0, [])

## 3. Adding the components
### 3.1 Defining symbols

The pyphs package is based on the `sympy` package to provides the symbolic manipulation of PHS structures. To declare symbols, we use the `Core.symbols` method. As an example, we declare below the symbols associated with the coil:

* the state $x_L$ (magnetic flux of the coil measured in Weber), and
* the parameter $L$ (coil inductance measured in Henry).

In [7]:
xL, L = RL.symbols(['xL', 'L'])
xL

xL

> Notice all symbols in `PyPHS` are assumed **real-valued only**. This is to alleviate complex solutions during the manipulation of expressions.

### 3.2 Defining expressions
Now, the variables `xL` and `L` are instances of the `Core.symbols` object, and expressions are defined with the standard `sympy` syntax. As an example, we define below the storage function associated with the coil $H_L(x_L)=\frac{x_L^2}{2L}$:

In [8]:
HL = xL**2/(2*L)

### 3.3 The `add_storages` method
To include a storage component to a `Core`, we make use of the `add_storages` method. As an example, the coil is added to the `Core` object as follows:

In [9]:
RL.add_storages(xL, HL)
RL.x

[xL]

In [10]:
RL.H

xL**2/(2*L)

### 3.4 The `add_dissipations` method
To include a dissipative component to a `Core`, we make use of the `add_dissipations` method. Recall the resistor is decribed by:
* the dissipative variable $w_R$ (resistor current),
* the parameter $R$ (electric resistance), and 
* the dissipation function $z_R(w_R)=R\,w_R$.

This component is added to the `Core` object as follows: 

In [11]:
wR, R = RL.symbols(['wR', 'R'])    # define sympy symbols
zR = R*wR                           # define sympy expression
RL.add_dissipations(wR, zR)        # add dissipation to the `core` object

Now, the dissipative variable of the `core` object includes `wR` only:

In [12]:
RL.w

[wR]

and the dissipation function is given by `zR` only:

In [13]:
RL.z

[R*wR]

### 3.5 The `add_ports` method
To include an external port to a `PHSCore` object, we make use of the `core.add_ports` method. Below, we define the external port with input $u=v_{\mathrm{out}}$ and output $y=i_{\mathrm{out}}$ (notice the symbols do not reflect the actual physical meaning of $u$ and $y$):

In [14]:
u1, y1, u2, y2 = RL.symbols(['v1', 'i1', 'v2', 'i2']) # define sympy symbols
RL.add_ports([u1, u2], [y1, y2])                  # add the port to the `core` object

In [15]:
RL.u

[v1, v2]

In [16]:
RL.y

[i1, i2]

## 4. Defining the interconnection structure
The interconnection structure of a PHS is defined by the matrix $\mathbf M$ structured as $$\mathbf M = \mathbf J- \mathbf R,$$ where 
* the skew-symmetric matrix $\mathbf J = \frac{1}{2}\left(\mathbf M- \mathbf M^\intercal\right) $ encodes the *conservative interconnection*, and 
* the symmetric positive definite matrix $\mathbf R = \frac{-1}{2}\left(\mathbf M + \mathbf M^\intercal\right)$ encodes the *dissipative interconnection*.

These matrices are decomposed in blocks as follows:
$$\mathbf M = \left( 
\begin{array}{lll}
\mathbf M_{\mathrm{xx}} & \mathbf M_{\mathrm{xw}} & \mathbf M_{\mathrm{xy}} \\
\mathbf M_{\mathrm{wx}} & \mathbf M_{\mathrm{ww}} & \mathbf M_{\mathrm{wy}} \\
\mathbf M_{\mathrm{yx}} & \mathbf M_{\mathrm{ww}} & \mathbf M_{\mathrm{yy}} \\
\end{array}\right), 
$$
$$
\mathbf J = \left( 
\begin{array}{lll}
\mathbf J_{\mathrm{xx}} & \mathbf J_{\mathrm{xw}} & \mathbf J_{\mathrm{xy}} \\
\mathbf J_{\mathrm{wx}} & \mathbf J_{\mathrm{ww}} & \mathbf J_{\mathrm{wy}} \\
\mathbf J_{\mathrm{yx}} & \mathbf J_{\mathrm{ww}} & \mathbf J_{\mathrm{yy}} \\
\end{array}\right), \quad\mathbf R = \left( 
\begin{array}{lll}
\mathbf R_{\mathrm{xx}} & \mathbf R_{\mathrm{xw}} & \mathbf R_{\mathrm{xy}} \\
\mathbf R_{\mathrm{wx}} & \mathbf R_{\mathrm{ww}} & \mathbf R_{\mathrm{wy}} \\
\mathbf R_{\mathrm{yx}} & \mathbf R_{\mathrm{ww}} & \mathbf R_{\mathrm{yy}} \\
\end{array}\right).$$

For the above description of the RLC circuit, the matrices are $$\mathbf M = \mathbf J = \left(\begin{array}{cc|c|c}
0 & -1 & -1 & -1\\ 
1 & 0 & 0 & 0 \\ \hline
1 & 0 & 0 & 0 \\  \hline
1 & 0 & 0 & 0
\end{array}\right),$$ 
that is, $\mathbf R=0$. 

> Notice `core.M` only is an attribute, all other matrices are accessed with *getters* (e.g. `core.J()`, `core.Jxx()`, `core.Rxy()`, `core.Mwy()`) and *setters* (e.g. `core.set_J()`, `core.set_Jxx()`, `core.set_Rxy()`, `core.set_Mwy()`).

First, we initialize the matrix $\mathbf M$ with zeros:

In [17]:
RL.init_M()

RL.M

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

In [18]:
RL.set_Mxx([0])
RL.M

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

In [19]:
Jxw = [[-1]]
RL.set_Jxw(Jxw)

RL.M

Matrix([
[0, -1, 0, 0],
[1,  0, 0, 0],
[0,  0, 0, 0],
[0,  0, 0, 0]])

> Notice the skew-symmetry (respectively symmetry) of $\mathbf J$ (respectively $\mathbf R$) is preserved with the *setters* `core.set_Jab` (respectively `core.set_Rab`), for $\mathtt{a, b}$ in $(\mathrm{x, w, y})^2$. Above, $\mathbf{J}_{\mathrm{wx}}=-\mathbf{J}_{\mathrm{xw}}^{\intercal}$ has been automatically updated.

In [20]:
RL.set_Jxy([[-1, -1]])
RL.M

Matrix([
[0, -1, -1, -1],
[1,  0,  0,  0],
[1,  0,  0,  0],
[1,  0,  0,  0]])

Then, the structure matrices are accessed as follows.

In [21]:
RL.M

Matrix([
[0, -1, -1, -1],
[1,  0,  0,  0],
[1,  0,  0,  0],
[1,  0,  0,  0]])

In [22]:
RL.J()

Matrix([
[  0, -1.0, -1.0, -1.0],
[1.0,    0,    0,    0],
[1.0,    0,    0,    0],
[1.0,    0,    0,    0]])

In [23]:
RL.R()

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

So that we recover $\mathbf{M}$ by substracting $\mathbf{J}$ (skew-symmetric part) and $\mathbf{R}$ (symmetric part).

# 5 How to connect two PHS?

This section describes how to connect two `Core` instances. 
We consider the example of the Thiele-Small model of a loudspeaker, that can be seen as the connexion between an RL circuit and a mass-spring-damper (MKA) system. The relation between electrical and mechanical domain is performed by a gyrator $BL$.  

### 5.1 The Mass-spring damper PHS


* $x_K$ is the spring position so that $v_K = \frac{\mathrm d x_K}{\mathrm d  t}$ is the spring velocity,  
* $x_M$ is the mass momentum so that $f_M = \frac{\mathrm d x_M}{\mathrm d  t}$ is the force applied on the mass,  
* $w_A= v_A$ is the damper velocity, 
* $y_3=v_3$ is the velocity of port #3 
* $u_3=f_3$ is the force applied on the port #3.

For this tutorial, the constitutive laws are:
* the storage function (Hamiltonian) $$H(x_M)=\frac{x_M^2}{2M}$$ so that the (linear) velocity of the mass is $v_M=\frac{\partial H}{\partial x_M}=\frac{x_M}{M}$ and $$H(x_K)=\frac{Kx_K^2}{2}$$ so that the (linear) force applied on the spring is $f_K = Kx_K$. 
* the linear dissipation function $z_A(w_A)= A \,w_A = f_A$ with $f_A$ force applied on the damper. 


The *Port-Hamiltonian Systems* (PHS) formalism of the one-port MKA system is

\begin{equation}
\begin{pmatrix}
\frac{dx_K}{dt} \\
\frac{dx_M}{dt} \\
\hline
w_A \\
\hline
v_3 
\end{pmatrix}
= 
\left(
\begin{array}{c c|c|c}
0 &1 &0& 0 \\
-1 &0 &-1 &-1 \\
\hline
0 &1 &0& 0 \\
\hline
0 &1& 0 &0
\end{array}
\right)
\begin{pmatrix}
\frac{dH}{dx_K} \\
\frac{dH}{dx_M} \\
\hline
z(w_A) \\
\hline
f_3
\end{pmatrix}
\end{equation}

We build the MKA Core object as previously for the RL circuit:

In [24]:
# Instantiate the MKA Core
MKA = Core()

# Define all symbols
xK, K, xM, M, wA, A, u3, v3 = MKA.symbols(['xK', 'K', 'xM', 'M', 'wA', 'A', 'f3', 'v3'])

# Define the constitutive laws
HK = (xK**2)*(K/2)
HM = (xM**2)/(2*M)
zA = wA*A 

# Add all the components
MKA.add_storages([xK, xM], HK + HM)
MKA.add_dissipations(wA, zA)
MKA.add_ports(u3, v3)

# Initialize the interconnexion matrix
MKA.init_M()

# It is possible to define M at once with a sympy.SparseMatrix
import sympy
MKA.M = sympy.SparseMatrix([[0, 1, 0, 0], [-1, 0, -1, -1], [0, 1, 0, 0], [0, 1, 0, 0]])

### 5.2 Connection of RL and MKA
Now we add both `Core` instances with the `+` operator to create a new PHS called SPK.

In [25]:
SPK = RL + MKA

All quantities from `RL` and `MKA` are concatenated into `SPK` and the energy is added:

In [26]:
SPK.x

[xL, xK, xM]

In [27]:
SPK.H

K*xK**2/2 + xM**2/(2*M) + xL**2/(2*L)

Then we define the connexion between RL and MKA with the `add_connector` method. It takes 3 arguments:
- The output of the the first PHS port (here $y_2$ for RL) 
- The output of the second PHS port (here $y_3$ for MKA)
- The physical law that connects both PHS (here $BL$)

In [28]:
# Define the symbol BL
BL = SPK.symbols('BL')

# Add the connector between the port #1 of RL and the port #3 of MKA
SPK.add_connector((SPK.y.index(RL.y[1]), SPK.y.index(MKA.y[0])), alpha = BL)

Then a second method `connect` is called to apply changes in the PHS structure:

In [29]:
# Apply the connector
SPK.connect()

The new interconnexion matrix M reads

In [30]:
SPK.M

Matrix([
[  0,  0, BL, -1,  0, -1],
[  0,  0,  1,  0,  0,  0],
[-BL, -1,  0,  0, -1,  0],
[  1,  0,  0,  0,  0,  0],
[  0,  0,  1,  0,  0,  0],
[  1,  0,  0,  0,  0,  0]])

that takes into account the connexion between electrical (RL) and mechanical (MKA) domains.


## 6. Setting the physical parameters values
The matching between the parameters symbols defined above (here `L`, `R`, `K`, `M`, `A` and `BL`) and their actual value is stored in the python dictionary `Core.subs`, with parameters symbols as the dictionary's keys and numerical values as the dictionary's values. Here, the physical parameters are
* $L=11\times 10^{-3}$H, 
* $R=5.7$ $\Omega$, 
* $K=4e7$ N/m, 
* $M = 19$g,
* $A = 0.406$ Ns/m

This is stored as follows:

In [31]:
# Physical parameters 
L_value = 11e-3     #11mH
R_value = 5.7   # 5.7 Ohm
K_value = 4e7 # N/m                         
M_value = 0.019 # 19g
A_value = 0.406 #Ns/m
BL_value = 2.99 #V/A

subs = {L: L_value,
        R: R_value,
        K: K_value, 
        M: M_value,
        A: A_value,
        BL:BL_value
       }

SPK.subs.update(subs)

It is possible to replace the constant parameters $BL$ by a function that depends on $x_K$:
$\mathrm{BLnl}(x_k) \triangleq \mathrm{B}\,e^{-x_k^2}$

In [32]:
# Define the new expression
B = SPK.symbols('B')
BLnl = B*sympy.exp(-(SPK.x[1])**2)

#Associate the expression to BL
SPK.substitute(subs={BL: BLnl})

# Print the changes in M
SPK.M

Matrix([
[             0,  0, B*exp(-xK**2), -1,  0, -1],
[             0,  0,             1,  0,  0,  0],
[-B*exp(-xK**2), -1,             0,  0, -1,  0],
[             1,  0,             0,  0,  0,  0],
[             0,  0,             1,  0,  0,  0],
[             1,  0,             0,  0,  0,  0]])

> Notice the option `selfsubs=True` can be passed to `SPK.apply_subs` to apply the substitution of all parameters in dictionary `core.subs`.

## 7. The `reduce_z` method
The resistive interconnection due to the linear physical laws in z can be incorporated in the matrix $\mathbf R$, so that the dimension of $z$ is reduced to that of its nonlinear part.

In [33]:
SPK.reduce_z()

ShapeError: 

> Notice the methods `split_linear()` and `reduce_z()` change the organization of the vectors $\mathbf x$ and $\mathbf w$ (compare the evaluation below with the original definition).

In [ ]:
SPK.R()

In [ ]:
SPK.M

In [ ]:
SPK.z

## 8. The `subsinverse` method
Some inverse relations occurs, for instance in the expression of $H$. They can be simplified thanks to the method `subsinverse`, that replaces inversed symbols by the same symbols with prefix 'inv', which is appended to the dictionary  'Core.subs.  

In [ ]:
SPK.subsinverse()

In [ ]:
SPK.H

In [ ]:
SPK.subs

## Bonus : Generate the $\LaTeX$ description
A `.tex` file containing a description of the system can now be generated with the `core.texwrite` command as follows:

In [ ]:
SPK.texwrite(path=None, title=None, authors=None, affiliations=None)

A [SPK.tex](/pyphs_outputs/RLC/core.tex) has been generated, the compilation of which yields the following [core.pdf](/pyphs_outputs/RLC/SPK.pdf). A valid file name and/or a document title can be specified.

## Bonus 2 : Print the PHS structure

In [ ]:
SPK.pprint()